In [1]:
import os
from turtle import mode
import numpy as np
import cv2
from keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm

In [2]:
# Target number - 48007
# Target class - Demented


# Define the path to your "Moderate demented" images
# moderate_demented_dir = 'Binary_ttv\\test\Demented'
moderate_demented_dir = 'Binary_Data\Demented'
# print(os.listdir(moderate_demented_dir)[0:5])

# Create a directory to save the augmented images
augmented_dir = 'Binary_Data\Demented\Augmented'
os.makedirs(augmented_dir, exist_ok=True)

# Initialize the ImageDataGenerator with augmentation settings
datagen = ImageDataGenerator(
    rescale=128./255,              # Rescale pixel values to [0, 1]
    brightness_range=[0.5, 1.5],   # Random brightness adjustment
    zoom_range=[0.8, 1.2],         # Random zooming
    fill_mode='nearest',           # Fill mode for filling pixels after augmentation
    horizontal_flip=True,          # Random horizontal flipping
    vertical_flip=True,            # Random vertical flipping
    rotation_range=30,             # Random rotations
    shear_range=0.3,               # Random shearing
)

# List all images in the "Moderate demented" directory
image_files = os.listdir(moderate_demented_dir)

# Set the target number of augmented images (e.g., 700)
target_count = 48007

In [3]:
# Loop until the target count is reached
generated_count = 0
print(f'Generating {target_count} augmented images...', end='\n\n')
with tqdm(total=target_count, desc="Augmentation Progress") as pbar:
    while generated_count < target_count:
        # Randomly select an image from the original dataset
        image_file = np.random.choice(image_files)
        image_path = os.path.join(moderate_demented_dir, image_file)

        # Load the image
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Ensure it's in RGB format

        # Generate augmented images using the data generator
        aug_iter = datagen.flow(np.expand_dims(image, axis=0), batch_size=1)
        augmented_images = [next(aug_iter)[0].astype(np.uint8) for _ in range(5)]  # Create 5 augmented images per original

        # Save the augmented images
        for i, aug_image in enumerate(augmented_images):
            aug_filename = f'augmented_{generated_count + i}.jpg'
            aug_image_path = os.path.join(augmented_dir, aug_filename)
            cv2.imwrite(aug_image_path, cv2.cvtColor(aug_image, cv2.COLOR_RGB2BGR))

        # Update the count of generated images
        generated_count += len(augmented_images)
        # if generated_count % 6000 == 0:
        #     print(f'{generated_count} images generated...')
        pbar.update(len(augmented_images))

print(f'Generated {generated_count} augmented images.')

Generating 48007 augmented images...



Augmentation Progress: 48010it [25:23, 31.52it/s]                           

Generated 48010 augmented images.
